# Handling Missing Values

It is important to understand that if the missing values are not handled properly by the analyst, then he/she may end up drawing an inaccurate inference about the data. Due to improper handling, the results obtained by the analyst will differ from ones where the missing values are present.

Let's examine a sample dataset that contains 1985 model import cars and the specification of each auto in terms of its various characteristics.  This data is stored as a CSV file on the Math@Work server.

In [2]:
import pandas as pd
autos = pd.read_csv('https://mathatwork.org/DATA/automobiles.csv')
print(autos.head())

          make fuel-type aspiration num-of-doors   body-style drive-wheels  \
0  alfa-romero       gas        std          two  convertible          rwd   
1  alfa-romero       gas        std          two  convertible          rwd   
2  alfa-romero       gas        std          two    hatchback          rwd   
3         audi       gas        std         four        sedan          fwd   
4         audi       gas        std         four        sedan          4wd   

  engine-location  wheel-base  length  width  height  curb-weight engine-type  \
0           front        88.6   168.8   64.1    48.8         2548        dohc   
1           front        88.6   168.8   64.1    48.8         2548        dohc   
2           front        94.5   171.2   65.5    52.4         2823        ohcv   
3           front        99.8   176.6   66.2    54.3         2337         ohc   
4           front        99.4   176.6   66.4    54.3         2824         ohc   

  num-of-cylinders  engine-size fuel-system 

We can use the following methods on the *autos* DataFrame to do a quick count of total missing values for each column in the dataset.

In [7]:
print(autos.isnull().sum())

make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type          12
num-of-cylinders      0
engine-size           0
fuel-system           0
compression-ratio     0
city-mpg              0
highway-mpg           0
price                 4
dtype: int64


In the above code, **isnull( )** goes through all rows and columns of the *autos* DataFrame and indicates *True* if the entry is null and *False* if it is not.  Then **.sum( )** gives us a total count of all *True* values returned in every column.

**METHOD 1:  Row deletion**

If a particular column has no more than 20% of missing values, then we can delete every data row with a missing value for that column. Let's do a count of the total number of rows in the *autos* DataFrame.

In [8]:
len(autos.index)

205

Nice.  Applying the **len( )** method to the index of the *autos* DataFrame does the trick.  For more information regarding the **len( )** method, review the Python for Data Visualizations workshop.

Let's apply the row deletion method to the *engine-type* column.  This column is a good candidate because no more than 75% of its values are missing.  To do this, we will use Pandas **.dropna( )** method and pass in the column name we want to delete the missing values from using the *subset* keyword.

In [10]:
autos_deleted = autos.dropna(subset=['engine-type'])
print(autos_deleted.isnull().sum())
len(autos_deleted.index)

make                 0
fuel-type            0
aspiration           0
num-of-doors         2
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
compression-ratio    0
city-mpg             0
highway-mpg          0
price                4
dtype: int64


193

Great job.  Notice that a quick count of total missing values for each column in the dataset reveals there are no more missing values in the *engine-type* column.  In addition, a count of the total number of remaining rows verifies that only the 12 rows with missing values in the *engine-type* column were deleted.

**METHOD 2:  Replacing With the Mean**

This strategy can only be applied to a column that has numeric data.  Here we calculate the mean of the entire column and use it to replace all the missing values in that column.

Let's apply this method to the *price* column.  This column is a good candidate because it is numeric. A quick check of this column's data type can be used to verify this.

In [12]:
print(autos.price.dtypes)

float64


The above code confirms that the *price* column is a double precision float (i.e., is numeric).  Now calculate the mean of the *price* column and replace all missing values in that column with this number.  To do this, we will use Pandas **.fillna( )** method applied to only the *price* column and pass in the calculated mean of the *price* column.  The keyword *inplace=True* allows the operation to be performed on the *autos* DataFrame itself and not a copy of it.

In [18]:
autos.price.fillna(autos.price.mean(), inplace=True)
print(autos.isnull().sum())
len(autos.index)

make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type          12
num-of-cylinders      0
engine-size           0
fuel-system           0
compression-ratio     0
city-mpg              0
highway-mpg           0
price                 0
dtype: int64


205

Notice that a quick count of total missing values for each column in the dataset reveals there are no more missing values in the *price* column. In addition, a count of the total number of rows in the *autos* DataFrame verifies there were no rows deleted in this process.

**METHOD 3:  Replacing With Similar Values**

This strategy should only be applied to a column that has categorical data.  In this method, missing values are replaced with a value from another row having the most similar attributes. 

Let's apply this method to the *num-of-doors* column. This column is a good candidate because it is categorical. A quick check of this column's data type can be used to verify this.

In [20]:
print(autos['num-of-doors'].dtypes)

object


Notice that because of the hyphens in the column name, *num-of-doors* had to be passed inside of quotes and then inside of brackets into the DataFrame.  Nonetheless, the above code verifies that *num-of-doors* is of type object (i.e., is a string or is categorical).

The goal is to find other rows in the DataFrame that are most similar to the rows that have missing values in the *num-of-doors* column.  We define *'similarity'* here as having the same *make* and *body-style*.  Presumably, if 2 cars have the same make and body styles they should have the same number of doors.
<br><br>
Let's filter on the rows that have missing values in the *num-of-doors* column.

In [21]:
missing = autos[autos['num-of-doors'].isnull()]
print(missing)

     make fuel-type aspiration num-of-doors body-style drive-wheels  \
27  dodge       gas      turbo          NaN      sedan          fwd   
63  mazda    diesel        std          NaN      sedan          fwd   

   engine-location  wheel-base  length  width  height  curb-weight  \
27           front        93.7   157.3   63.8    50.6         2191   
63           front        98.8   177.8   66.5    55.5         2443   

   engine-type num-of-cylinders  engine-size fuel-system  compression-ratio  \
27         ohc             four           98        mpfi                7.6   
63         ohc             four          122         idi               22.7   

    city-mpg  highway-mpg    price  
27        24           30   8558.0  
63        36           42  10795.0  


We can now see that the makes of cars with missing *num-of-doors* values are dodge and mazda.  The body-styles are both sedan.  Since similarity was defined as having the same *make* and *body-style*, let's now filter on *make=dodge OR mazda* AND *body-style=sedan*.

In [36]:
autos_filtered = autos[((autos.make == 'dodge') | (autos.make == 'mazda')) & (autos['body-style'] == 'sedan')]
print(autos_filtered[['make','num-of-doors','body-style']])

     make num-of-doors body-style
25  dodge         four      sedan
26  dodge         four      sedan
27  dodge          NaN      sedan
53  mazda         four      sedan
54  mazda         four      sedan
60  mazda         four      sedan
62  mazda         four      sedan
63  mazda          NaN      sedan
65  mazda         four      sedan
66  mazda         four      sedan


Recall from the Python for Data Science workshop that | is the symbol for OR and & is the symbol for AND.  Examining the *make* and *body-style* columns, the missing *num-of-doors* is more than likely four.  Let's replace the missing values in this column with four.

In [37]:
autos['num-of-doors'].fillna('four', inplace=True)
print(autos.isnull().sum())
len(autos.index)

make                  0
fuel-type             0
aspiration            0
num-of-doors          0
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type          12
num-of-cylinders      0
engine-size           0
fuel-system           0
compression-ratio     0
city-mpg              0
highway-mpg           0
price                 0
dtype: int64


205

Notice that a quick count of total missing values for each column in the dataset reveals there are no more missing values in the *num-of-doors* column. In addition, a count of the total number of rows in the autos DataFrame verifies there were no rows deleted in this process.

Whichever method or methods you choose to apply is up to you.  However, you must ensure that the method is appropriate for the given situation.  Keep in mind, though, that if a column has more than 20% of missing data, that column should be removed and NOT be used for analysis.

### Exercise

Let's examine a sample dataset that contains 150 observations of iris plants and their various characteristics. This data is stored as a CSV file on the Math@Work server.

In [57]:
iris = pd.read_csv('https://mathatwork.org/DATA/iris.csv')
print(iris.head())

   sepal_length  sepal_width  petal_length  petal_width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


Though it may not be clear now, examining the unique class types will be helpful later.  Take a look at a listing of unique elements in the *class* column.

In [58]:
print(iris.groupby('class').first())

                 sepal_length  sepal_width  petal_length  petal_width
class                                                                
Iris-setosa               5.1          3.5           1.4          0.2
Iris-versicolor           7.0          3.2           4.7          1.4
Iris-virginica            6.3          3.3           6.0          2.5


**1)** Do a quick count of the total missing values for each column in the dataset.

**2)** Apply METHOD 1 to the *sepal_width* column.  Remember to verify no missing values remain in the *sepal_width* column and that the correct number of rows were deleted.

Take a look at the row data for missing values in the *sepal_length* column.

In [61]:
missing = iris[iris.sepal_length.isnull()]
print(missing)

    sepal_length  sepal_width  petal_length  petal_width        class
26           NaN          3.4           1.6          0.4  Iris-setosa


You see that this missing *sepal_length* belongs to the Iris-setosa class.  Recall there are 3 unique classes.  We may get better results in the application of METHOD 2 by replacing this missing value with the mean of its class and not the mean of the entire *sepal_length* column.

**3)** Apply METHOD 2 to the *sepal_length* column altering the formula to calculate the mean for the class of the missing value and not the mean of the entire *sepal_length* column.  Remember to verify no missing values remain in the *sepal_length* column and that no rows were deleted.

Take a look at the row data for missing values in the *class* column.

In [65]:
missing = iris[iris['class'].isnull()]
print(missing)

    sepal_length  sepal_width  petal_length  petal_width class
24           4.8          3.4           1.9          0.2   NaN
68           6.2          2.2           4.5          1.5   NaN


**4)** Apply METHOD 3 to the *class* column.  Define 'similarity' as the class that is closest to the missing *class* in mean *sepal_length*, mean *sepal_width*, mean *petal_length* and mean *petal_width*.

Run the following code to get the means for each *class*.

In [ ]:
means = iris.groupby('class').mean()
print(means)

Run the following code to replace the first missing value (index 24) in the *class* column with the estimated class Iris-setosa.

In [ ]:
iris['class'].fillna('Iris-setosa', limit=1, inplace=True)
print(iris.isnull().sum())
len(iris.index)

The above code verifies only 1 missing value was filled.  
<br>
Fill in the last missing value with your estimated *class* using similar code as above.  Remember to verify no missing values remain in the *class* column and that no rows were deleted.